In [1]:
import setup  # notebooks/setup.py
setup.init()

In [2]:
from my_permit import permit_client
permit = permit_client

In [3]:
await permit.api.users.list()

PaginatedResultUserRead(data=[UserRead(key='1', id=UUID('1a201c07-9452-4b4b-bc5d-b9ba2678567f'), organization_id=UUID('e1e76d58-7a6c-4a4b-97fd-0f40c19db09c'), project_id=UUID('834a942b-2bf0-4a5b-bb7b-136dac09046c'), environment_id=UUID('3e48b81c-755e-47d3-8bf4-68aa2305b11e'), associated_tenants=[], roles=[], created_at=datetime.datetime(2025, 7, 23, 15, 4, 49, tzinfo=datetime.timezone.utc), updated_at=datetime.datetime(2025, 7, 23, 15, 10, 10, tzinfo=datetime.timezone.utc), email=None, first_name='pashinator', last_name=None, attributes=None), UserRead(key='3', id=UUID('62d9fc80-d7cb-4ac7-8391-b4ae568ac5a3'), organization_id=UUID('e1e76d58-7a6c-4a4b-97fd-0f40c19db09c'), project_id=UUID('834a942b-2bf0-4a5b-bb7b-136dac09046c'), environment_id=UUID('3e48b81c-755e-47d3-8bf4-68aa2305b11e'), associated_tenants=[], roles=[], created_at=datetime.datetime(2025, 7, 23, 15, 4, 49, tzinfo=datetime.timezone.utc), updated_at=datetime.datetime(2025, 7, 23, 15, 10, 11, tzinfo=datetime.timezone.utc), e

In [4]:
from django.contrib.auth import get_user_model

User = get_user_model()

In [5]:
for u in User.objects.all():
    print(u.id, u.username)
    user_data = {
        "key": f"{u.id}",
        "first_name": f"{u.username}"
    }
    response = await permit.api.users.sync(user_data)
    print(response)

1 pashinator
key='1' id=UUID('1a201c07-9452-4b4b-bc5d-b9ba2678567f') organization_id=UUID('e1e76d58-7a6c-4a4b-97fd-0f40c19db09c') project_id=UUID('834a942b-2bf0-4a5b-bb7b-136dac09046c') environment_id=UUID('3e48b81c-755e-47d3-8bf4-68aa2305b11e') associated_tenants=[] roles=[] created_at=datetime.datetime(2025, 7, 23, 15, 4, 49, tzinfo=datetime.timezone.utc) updated_at=datetime.datetime(2025, 7, 23, 15, 10, 10, tzinfo=datetime.timezone.utc) email=None first_name='pashinator' last_name=None attributes=None
3 steve_hunter
key='3' id=UUID('62d9fc80-d7cb-4ac7-8391-b4ae568ac5a3') organization_id=UUID('e1e76d58-7a6c-4a4b-97fd-0f40c19db09c') project_id=UUID('834a942b-2bf0-4a5b-bb7b-136dac09046c') environment_id=UUID('3e48b81c-755e-47d3-8bf4-68aa2305b11e') associated_tenants=[] roles=[] created_at=datetime.datetime(2025, 7, 23, 15, 4, 49, tzinfo=datetime.timezone.utc) updated_at=datetime.datetime(2025, 7, 23, 15, 10, 11, tzinfo=datetime.timezone.utc) email=None first_name='steve_hunter' last_na

In [6]:
await permit.check("user_id:1", "read", "document")

False

In [7]:
resources = [
    {
        "name": "Document",
        "key": "document",
        "actions": {
            "read": {},
            "create": {},
            "update": {},
            "delete": {},
        }
    },
    {
        "name": "Movie Discovery Resource",
        "key": "movie_disovery",
        "actions": {
            "search": {},
            "detail": {},
        }
    }
]

In [8]:
from permit import PermitApiError

async def get_or_create_resource(resource_data:dict):
    resource_key = resource_data.get('key')
    try:
        _resource = await permit.api.resources.get(resource_key)
    except PermitApiError as e:
        if e.status_code == 404:
            _resource = await permit.api.resources.create(resource_data)
        else:
            raise e
    return _resource

In [9]:
for resource_data in resources:
    response = await get_or_create_resource(resource_data)
    print(response)

key='document' id=UUID('2df3265a-512d-41ef-a25d-9ec00e456feb') organization_id=UUID('e1e76d58-7a6c-4a4b-97fd-0f40c19db09c') project_id=UUID('834a942b-2bf0-4a5b-bb7b-136dac09046c') environment_id=UUID('3e48b81c-755e-47d3-8bf4-68aa2305b11e') created_at=datetime.datetime(2025, 7, 27, 7, 38, 52, tzinfo=datetime.timezone.utc) updated_at=datetime.datetime(2025, 7, 27, 7, 38, 52, tzinfo=datetime.timezone.utc) name='Document' urn='prn:pashinator:default:document' description=None actions={'read': ActionBlockRead(name='read', description=None, attributes=None, v1compat_path=None, v1compat_name=None, id=UUID('a02defd2-2007-45e9-bc2d-54936c18e75a'), key='read'), 'create': ActionBlockRead(name='create', description=None, attributes=None, v1compat_path=None, v1compat_name=None, id=UUID('f3920441-4d86-497f-ac2e-e27779b6a8e1'), key='create'), 'update': ActionBlockRead(name='update', description=None, attributes=None, v1compat_path=None, v1compat_name=None, id=UUID('97f3f4b4-6fa3-4225-8ed4-e6ad7b6be7f

In [10]:
roles = [
    {
        "name": "Manager",
        "key": "manager",
        "permissions": [
            "document:read",
            "document:create",
            "document:update",
            "document:delete",
            "movie_disovery:search",
            "movie_disovery:detail"
        ]
    }, 
    {
        "name": "Viewer",
        "key": "viewer",
        "permissions": [
            "document:read",
            "movie_disovery:search",
            "movie_disovery:detail"
        ]
    }
]

In [11]:
from permit import PermitApiError

async def get_or_create_role(role_data:dict):
    _key = role_data.get('key')
    try:
        _resource = await permit.api.roles.get(_key)
    except PermitApiError as e:
        if e.status_code == 404:
            _resource = await permit.api.roles.create(role_data)
        else:
            raise e
    return _resource

In [12]:
for role_data in roles:
    response = await get_or_create_role(role_data)
    print(response)

name='Manager' description=None permissions=['movie_disovery:detail', 'document:create', 'document:update', 'document:delete', 'movie_disovery:search', 'document:read'] attributes={} extends=[] granted_to=None v1compat_settings=None v1compat_attributes=None key='manager' id=UUID('63256f4c-487c-4302-a5d8-634ebc8efe2c') organization_id=UUID('e1e76d58-7a6c-4a4b-97fd-0f40c19db09c') project_id=UUID('834a942b-2bf0-4a5b-bb7b-136dac09046c') environment_id=UUID('3e48b81c-755e-47d3-8bf4-68aa2305b11e') created_at=datetime.datetime(2025, 7, 27, 7, 38, 55, tzinfo=datetime.timezone.utc) updated_at=datetime.datetime(2025, 7, 27, 7, 38, 55, tzinfo=datetime.timezone.utc)
name='viewer' description=None permissions=[] attributes={} extends=[] granted_to=None v1compat_settings=None v1compat_attributes=None key='viewer' id=UUID('46177ecd-e8c9-4ce8-b929-b51b9a1e2fa3') organization_id=UUID('e1e76d58-7a6c-4a4b-97fd-0f40c19db09c') project_id=UUID('834a942b-2bf0-4a5b-bb7b-136dac09046c') environment_id=UUID('3e4

In [14]:
user = User.objects.all().first()

In [15]:
user.id

1

In [16]:
response = await permit.api.users.assign_role({
    "user": f"{user.id}",
    "role": "admin",
    "tenant": "default"
})
response

RoleAssignmentRead(id=UUID('fc1abe96-8310-4e55-abe7-28adef405ade'), user='1', role='admin', tenant='default', resource_instance=None, resource_instance_id=None, user_id=UUID('1a201c07-9452-4b4b-bc5d-b9ba2678567f'), role_id=UUID('44195710-e43e-490d-8f28-bdc5337a28b5'), tenant_id=UUID('2e80a657-612c-4ed4-aab5-294713cdd314'), organization_id=UUID('e1e76d58-7a6c-4a4b-97fd-0f40c19db09c'), project_id=UUID('834a942b-2bf0-4a5b-bb7b-136dac09046c'), environment_id=UUID('3e48b81c-755e-47d3-8bf4-68aa2305b11e'), created_at=datetime.datetime(2025, 7, 27, 7, 44, 30, tzinfo=datetime.timezone.utc))

In [ ]:
await permit.check(f"{user.id}", "read", "document")

False

In [ ]:
from langgraph.checkpoint.memory import InMemorySaver 
from ai.supervisors import get_supervisor
import uuid 

checkpointer = InMemorySaver()
supervisor = get_supervisor(model="gpt-4o", checkpointer=checkpointer)

In [ ]:
doc_id = 24
user_id = user.id
user_id

In [ ]:
config = {"configurable": {"user_id": user_id, "thread_id": uuid.uuid4()}}

response = supervisor.invoke(
    {"messages": 
        [
            {"role": "user", "content": "what are my recent documents?"}
        ]
    },
    config
)

In [ ]:
response['messages'][-1].content

In [ ]:
response = supervisor.invoke(
    {"messages": 
        [
            {"role": "user", "content": f"Delete any random document by id"}
        ]
    },
    config
)
response['messages'][-1].content